In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import re
from datetime import datetime
from selenium.webdriver.common.keys import Keys

In [4]:
#Inputs to search
search_box_text = 'sports shoes for women'
website_link = 'https://www.flipkart.com/'

#initiating the browser
#session start time
session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} ---------------------------> ")


#starting the browser
driver = webdriver.Chrome()
driver.get(website_link)
driver.maximize_window()

print('Waiting for search input...')
search_input = WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[autocomplete="off"]')))

print('Typing in search input...')
search_input.send_keys(search_box_text)

print('Submitting search form...')
search_input.send_keys(Keys.RETURN)

print('Waiting for search results...')
WebDriverWait(driver, 120).until( EC.presence_of_element_located((By.CSS_SELECTOR, '[target="_blank"]')) )

print('Collecting pagination links...')


#we want first 25 pages [pagination link]  [1000 Products]
#logic: Let's get the first page pagination link and append the number in the end for 25 pages and store in a list
all_pagination_links =[]

first_page = driver.find_elements(By.CSS_SELECTOR, 'nav a')[0]
first_page_link = first_page.get_attribute('href')
all_pagination_links.append(first_page_link)

for i in range(2, 26):
    new_pagination_link = first_page_link[: -1] + str(i)
    all_pagination_links.append(new_pagination_link)

print('Pagination Links Count:', len(all_pagination_links))
print("All Pagination Links: ", all_pagination_links)


print("Collecting Product Detail Page Links")
all_product_links = []

for link in all_pagination_links:
    driver.get(link)
    # Wait for the page to load by checking document.readyState
    WebDriverWait(driver, 120).until(lambda d: d.execute_script('return document.readyState') == 'complete')

    #wait until elements located
    WebDriverWait(driver, 120).until(EC.presence_of_element_located((By.CLASS_NAME, 'rPDeLR')))

    all_products = driver.find_elements(By.CLASS_NAME, 'rPDeLR')
    all_links = [element.get_attribute('href') for element in all_products]

    print(f"{link} Done ------>")

    all_product_links.extend(all_links)

print('All Product Detail Page Links Captured: ', len(all_product_links))


# Creating a DataFrame from the list
df_product_links = pd.DataFrame(all_product_links, columns=['product_links'])
#remove any duplicates
df_product_links = df_product_links.drop_duplicates(subset=['product_links'])

print("Total Product Detail Page Links", len(df_product_links))
df_product_links.to_csv('flipkart_product_links.csv', index = False)

driver.close()
session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} ---------------------------> ")

Session Start Time: 23:43:45.580946 ---------------------------> 
Waiting for search input...
Typing in search input...
Submitting search form...
Waiting for search results...
Pagination Links Count: 25
All Pagination Links:  ['https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=1', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=2', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=3', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=4', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=5', 'https://www.flipkart.com/search?q=sports+shoes+for+women&otracker=search

TimeoutException: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7b43788d5
	0x7ff7b4378930
	0x7ff7b415165d
	0x7ff7b41a9a33
	0x7ff7b41a9d3c
	0x7ff7b41fdf67
	0x7ff7b41fac97
	0x7ff7b419ac29
	0x7ff7b419ba93
	0x7ff7b4690620
	0x7ff7b468af60
	0x7ff7b46a96c6
	0x7ff7b4395dd4
	0x7ff7b439ed7c
	0x7ff7b4381ff4
	0x7ff7b43821a5
	0x7ff7b4367ed2
	0x7ff922a7e8d7
	0x7ff92324c53c


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pandas as pd
import time

# ==============================
# INPUTS
# ==============================
search_box_text = "sports shoes for women"
website_link = "https://www.flipkart.com/"
MAX_PAGES = 25

# ==============================
# SESSION START
# ==============================
session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} ---------------------------> ")

# ==============================
# START BROWSER
# ==============================
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
driver.get(website_link)

wait = WebDriverWait(driver, 30)

# ==============================
# CLOSE LOGIN POPUP (IMPORTANT)
# ==============================
try:
    wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='✕']"))
    ).click()
    print("Login popup closed")
except:
    print("No login popup")

# ==============================
# SEARCH PRODUCT
# ==============================
print("Waiting for search box...")
search_input = wait.until(
    EC.presence_of_element_located((By.NAME, "q"))
)

search_input.send_keys(search_box_text)
search_input.send_keys(Keys.RETURN)

# ==============================
# WAIT FOR RESULTS
# ==============================
wait.until(
    EC.presence_of_element_located((By.XPATH, "//a[contains(@href,'/p/')]"))
)

# ==============================
# SCRAPE PRODUCT LINKS
# ==============================
all_product_links = []
current_page = 1

while current_page <= MAX_PAGES:
    print(f"Scraping Page {current_page}")

    try:
        wait.until(
            EC.presence_of_element_located((By.XPATH, "//a[contains(@href,'/p/')]"))
        )

        products = driver.find_elements(By.XPATH, "//a[contains(@href,'/p/')]")
        links = [p.get_attribute("href") for p in products if p.get_attribute("href")]

        all_product_links.extend(links)
        print(f"Page {current_page} -> {len(links)} products collected")

    except TimeoutException:
        print(f"Page {current_page} skipped (no products found)")

    # ==============================
    # CLICK NEXT PAGE
    # ==============================
    try:
        next_btn = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[span[text()='Next']]"))
        )
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        current_page += 1
    except:
        print("No more pages available")
        break

# ==============================
# SAVE TO CSV
# ==============================
df = pd.DataFrame(set(all_product_links), columns=["product_links"])
print("Total Unique Product Links:", len(df))

df.to_csv("flipkart_product_links.csv", index=False)

# ==============================
# CLOSE SESSION
# ==============================
driver.quit()

session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} ---------------------------> ")


Session Start Time: 23:50:13.487726 ---------------------------> 
No login popup
Waiting for search box...
Scraping Page 1
Page 1 -> 125 products collected
Scraping Page 2
Page 2 -> 125 products collected
Scraping Page 3
Page 3 -> 125 products collected
Scraping Page 4
Page 4 -> 125 products collected
Scraping Page 5
Page 5 -> 125 products collected
Scraping Page 6
Page 6 -> 125 products collected
Scraping Page 7
Page 7 -> 125 products collected
Scraping Page 8
Page 8 -> 125 products collected
Scraping Page 9
Page 9 -> 125 products collected
Scraping Page 10
Page 10 -> 125 products collected
Scraping Page 11
Page 11 -> 125 products collected
Scraping Page 12
Page 12 -> 125 products collected
Scraping Page 13
Page 13 -> 125 products collected
Scraping Page 14
Page 14 -> 125 products collected
Scraping Page 15
Page 15 -> 125 products collected
Scraping Page 16
Page 16 -> 125 products collected
Scraping Page 17
Page 17 -> 125 products collected
Scraping Page 18
Page 18 -> 125 products col

In [7]:
#session start time
session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} ---------------------------> ")


#reading the csv file which contain all product links
df_product_links = pd.read_csv("flipkart_product_links.csv")

# Remove the below line to scrap all the products. For demonstration purpose we are scraping only 10 products
# df_product_links = df_product_links.head(10)

all_product_links = df_product_links['product_links'].tolist()
print("Collecting Individual Product Detail Information")

#starting the browser
driver = webdriver.Chrome()


complete_product_details = []
unavailable_products = []
successful_parsed_urls_count = 0
complete_failed_urls_count = 0
for product_page_link in all_product_links:

    try:
        driver.get(product_page_link)

        # Wait for the page to load by checking document.readyState
        WebDriverWait(driver, 120).until(lambda d: d.execute_script('return document.readyState') == 'complete')

        WebDriverWait(driver, 120).until( EC.presence_of_element_located((By.CSS_SELECTOR, '[target="_blank"]')))

        #checking if product is available or not
        try:
            product_status =  driver.find_element(By.CLASS_NAME, 'Z8JjpR').text
            if product_status == 'Currently Unavailable' or product_status == 'Sold Out':
                unavailable_products.append(product_page_link)
                successful_parsed_urls_count += 1
                print(f"URL {successful_parsed_urls_count} completed --->")
        except:
            pass

        #brand
        brand =  driver.find_element(By.CLASS_NAME, 'mEh187').text

        #title
        title = driver.find_element(By.CLASS_NAME, 'VU-ZEz').text
        title = re.sub(r'\s*\([^)]*\)', '', title)  #removing data withing parenthesis (color information)

        #price
        price = driver.find_element(By.CLASS_NAME, 'Nx9bqj').text
        price = re.findall(r'\d+', price)
        price = ''.join(price)

        # Discount
        try:
            discount = driver.find_element(By.CLASS_NAME, 'UkUFwK').text
            discount = re.findall(r'\d+', discount)
            discount = ''.join(discount)
            discount = int(discount) / 100
        except:
            discount = ''

        #for a new product, there will be no avg_rating and total_ratings
        try:
            product_review_status = driver.find_element(By.CLASS_NAME, 'E3XX7J').text
            if product_review_status == 'Be the first to Review this product':
                avg_rating = ''
                total_ratings = ''
        except:
            avg_rating = driver.find_element(By.CLASS_NAME, 'XQDdHH').text
            total_ratings = driver.find_element(By.CLASS_NAME, 'Wphh3N').text.split(' ')[0]
            #remove the special character
            if ',' in total_ratings:
                total_ratings = int(total_ratings.replace(',', ''))
            else:
                total_ratings = int(total_ratings)

        successful_parsed_urls_count += 1
        print(f"URL {successful_parsed_urls_count} completed *******")
        complete_product_details.append([product_page_link, title, brand, price, discount, avg_rating, total_ratings])
    except Exception as e:
        print(f"Failed to establish a connection for URL {product_page_link}:  {e}")
        unavailable_products.append(product_page_link)
        complete_failed_urls_count += 1
        print(f"Failed URL Count {complete_failed_urls_count}")


#create pandas dataframe
df = pd.DataFrame(complete_product_details, columns = ['product_link', 'title', 'brand', 'price', 'discount', 'avg_rating', 'total_ratings'])
#duplicates processing
df_duplicate_products = df[df.duplicated(subset=['brand', 'price', 'discount', 'avg_rating', 'total_ratings'])]
df = df.drop_duplicates(subset=['brand', 'price', 'discount', 'avg_rating', 'total_ratings'])
#unavailable products
df_unavailable_products = pd.DataFrame(unavailable_products, columns=['link'])


#prining the stats
print("Total product pages scrapped: ", len(all_product_links))
print("Final Total Products: ", len(df))
print("Total Unavailable Products : ", len(df_unavailable_products))
print("Total Duplicate Products: ", len(df_duplicate_products))


#saving all the files
df.to_csv('flipkart_product_data.csv', index = False)
df_unavailable_products.to_csv('unavailable_products.csv', index = False)
df_duplicate_products.to_csv('duplicate_products.csv', index = False)


driver.close()
session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} ---------------------------> ")

Session Start Time: 23:57:58.443799 ---------------------------> 
Failed to establish a connection for URL https://www.flipkart.com/sparx-sl-194-running-shoes-women/p/itma10aaf9018f67?pid=SHOGCPSEZZUJU2W8&lid=LSTSHOGCPSEZZUJU2W8BKXTGJ&marketplace=FLIPKART&q=sports+shoes+for+women&store=osp%2Fiko%2Fd20&srno=s_22_874&otracker=search&otracker1=search&fm=Search&iid=2cc9926f-8ae9-4e0c-a246-b42d102b9363.SHOGCPSEZZUJU2W8.SEARCH&ppt=sp&ppn=sp&ssid=wimh7z605c0000001768414849510&qH=32e27b1148959538:  Message: no such element: Unable to locate element: {"method":"css selector","selector":".mEh187"}
  (Session info: chrome=143.0.7499.193); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7b43788d5
	0x7ff7b4378930
	0x7ff7b415165d
	0x7ff7b41a9a33
	0x7ff7b41a9d3c
	0x7ff7b41fdf67
	0x7ff7b41fac97
	0x7ff7b419ac29
	0x7ff7b419ba93
	0x7ff7b46906

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime
import pandas as pd
import re
import time

# ==============================
# SESSION START
# ==============================
session_start_time = datetime.now().time()
print(f"Session Start Time: {session_start_time} ---------------------------> ")

# ==============================
# READ PRODUCT LINKS
# ==============================
df_product_links = pd.read_csv("flipkart_product_links.csv")
all_product_links = df_product_links['product_links'].dropna().tolist()

print("Total URLs to scrape:", len(all_product_links))

# ==============================
# START BROWSER
# ==============================
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 30)

# ==============================
# CLOSE LOGIN POPUP
# ==============================
driver.get("https://www.flipkart.com/")
try:
    wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[text()='✕']"))
    ).click()
    print("Login popup closed")
except:
    pass

# ==============================
# STORAGE
# ==============================
complete_product_details = []
unavailable_products = []
failed_products = []

successful_count = 0

# ==============================
# SCRAPE PRODUCTS
# ==============================
for idx, product_page_link in enumerate(all_product_links, start=1):
    try:
        driver.get(product_page_link)

        # Wait for product title (stable)
        wait.until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(@class,'B_NuCI')]"))
        )

        # ---------- UNAVAILABLE CHECK ----------
        try:
            unavailable_text = driver.find_element(
                By.XPATH, "//*[contains(text(),'Currently Unavailable') or contains(text(),'Sold Out')]"
            ).text
            unavailable_products.append(product_page_link)
            print(f"[{idx}] Unavailable")
            continue
        except:
            pass

        # ---------- BRAND ----------
        try:
            brand = driver.find_element(
                By.XPATH, "//span[contains(@class,'G6XhRU')]"
            ).text
        except:
            brand = ""

        # ---------- TITLE ----------
        title = driver.find_element(
            By.XPATH, "//span[contains(@class,'B_NuCI')]"
        ).text
        title = re.sub(r'\s*\([^)]*\)', '', title)

        # ---------- PRICE ----------
        try:
            price = driver.find_element(
                By.XPATH, "//div[contains(@class,'_30jeq3')]"
            ).text
            price = ''.join(re.findall(r'\d+', price))
        except:
            price = ""

        # ---------- DISCOUNT ----------
        try:
            discount = driver.find_element(
                By.XPATH, "//div[contains(text(),'% off')]"
            ).text
            discount = int(re.findall(r'\d+', discount)[0]) / 100
        except:
            discount = ""

        # ---------- RATINGS ----------
        try:
            avg_rating = driver.find_element(
                By.XPATH, "//div[contains(@class,'_3LWZlK')]"
            ).text
        except:
            avg_rating = ""

        try:
            total_ratings = driver.find_element(
                By.XPATH, "//span[contains(text(),'Ratings')]"
            ).text
            total_ratings = int(re.findall(r'\d+', total_ratings.replace(',', ''))[0])
        except:
            total_ratings = ""

        complete_product_details.append([
            product_page_link, title, brand, price,
            discount, avg_rating, total_ratings
        ])

        successful_count += 1
        print(f"[{idx}] Success ({successful_count})")

    except Exception as e:
        failed_products.append(product_page_link)
        print(f"[{idx}] FAILED → {e}")

# ==============================
# DATAFRAMES
# ==============================
df = pd.DataFrame(
    complete_product_details,
    columns=['product_link', 'title', 'brand', 'price', 'discount', 'avg_rating', 'total_ratings']
)

df_duplicates = df[df.duplicated(subset=['brand', 'price', 'discount', 'avg_rating', 'total_ratings'])]
df = df.drop_duplicates(subset=['brand', 'price', 'discount', 'avg_rating', 'total_ratings'])

df_unavailable = pd.DataFrame(unavailable_products, columns=['link'])
df_failed = pd.DataFrame(failed_products, columns=['link'])

# ==============================
# SAVE FILES
# ==============================
df.to_csv('flipkart_product_data.csv', index=False)
df_unavailable.to_csv('unavailable_products.csv', index=False)
df_duplicates.to_csv('duplicate_products.csv', index=False)
df_failed.to_csv('failed_products.csv', index=False)

# ==============================
# STATS
# ==============================
print("\n========= SUMMARY =========")
print("Total URLs:", len(all_product_links))
print("Success:", len(df))
print("Unavailable:", len(df_unavailable))
print("Duplicates:", len(df_duplicates))
print("Failed:", len(df_failed))

driver.quit()

session_end_time = datetime.now().time()
print(f"Session End Time: {session_end_time} ---------------------------> ")


Session Start Time: 00:42:36.837339 ---------------------------> 
Total URLs to scrape: 1000
[1] FAILED → Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7b43788d5
	0x7ff7b4378930
	0x7ff7b415165d
	0x7ff7b41a9a33
	0x7ff7b41a9d3c
	0x7ff7b41fdf67
	0x7ff7b41fac97
	0x7ff7b419ac29
	0x7ff7b419ba93
	0x7ff7b4690620
	0x7ff7b468af60
	0x7ff7b46a96c6
	0x7ff7b4395dd4
	0x7ff7b439ed7c
	0x7ff7b4381ff4
	0x7ff7b43821a5
	0x7ff7b4367ed2
	0x7ff922a7e8d7
	0x7ff92324c53c

[2] FAILED → Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7b43788d5
	0x7ff7b4378930
	0x7ff7b415165d
	0x7ff7b41a9a33
	0x7ff7b41a9d3c
	0x7ff7b41fdf67
	0x7ff7b41fac97
	0x7ff7b419ac29
	0x7ff7b419ba93
	0x7ff7b4690620
	0x7ff7b468af60
	0x7ff7b46a96c6
	0x7ff7b4395dd4
	0x7ff7b439ed7c
	0x7ff7b4381ff4
	0x7ff7b43821a5
	0x7ff7b4367ed2
	0x7ff922a7e8d7
	0x7ff92324c53c

[3] FAILED → Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff7b43788d5
	0x7ff7b4378930
	

KeyboardInterrupt: 